In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sklearn import set_config; set_config(display='diagram')
from package.etl import *
from package.train_baseline import TrainBaseline
from package.train_city import TrainCity
from package.train_store import TrainStore
from package.train_family import TrainFamily
import pandas as pd
import numpy as np
import requests
import joblib
import json

# Train Baseline (Original)

In [3]:
X_train, y_train, X_predict, pipe, preprocessor = TrainBaseline()

In [4]:
X_predict.head(12)

,year,month,family,store_nbr,onpromotion
0,2017,8,AUTOMOTIVE,1,0
1,2017,8,AUTOMOTIVE,2,0
2,2017,8,AUTOMOTIVE,3,0
3,2017,8,AUTOMOTIVE,4,0
4,2017,8,AUTOMOTIVE,5,0
5,2017,8,AUTOMOTIVE,6,0
6,2017,8,AUTOMOTIVE,7,0
7,2017,8,AUTOMOTIVE,8,0
8,2017,8,AUTOMOTIVE,9,0
9,2017,8,AUTOMOTIVE,10,2


In [5]:
job = joblib.load('model_baseline.joblib')
job

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['year', 'month', 'store_nbr',
                                                   'family'])])),
                ('linearregression', LinearRegression())])

In [6]:
job.score(X_train,y_train)

0.6927092577492131

In [7]:
p = job.predict(X_predict).tolist()
p[0:12]

[6.706438899040222,
 7.0134095549583435,
 7.783157706260681,
 6.83639532327652,
 6.796813130378723,
 7.000763237476349,
 7.152514815330505,
 7.365639269351959,
 6.961523175239563,
 5.6811699867248535,
 6.825519561767578,
 5.7159886956214905]

In [8]:
p = [np.around(np.exp(i),decimals=2) for i in p]
p = np.array(p) 
p[0:12]

array([ 817.65, 1111.44, 2399.84,  931.13,  894.99, 1097.47, 1277.31,
       1580.73, 1055.24,  293.29,  921.05,  303.68])

In [9]:
year=['2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017']
month=['8','8','8','8','8','8','8','8','8','8','8','8']
family=['AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE','AUTOMOTIVE']
store_nbr=['1','2','3','4','5','6','7','8','9','10','11','12']
onpromotion=['0','0','0','0','0','0','0','0','0','0','0','0']

url = 'http://127.0.0.1:8000/predict-baseline-year'
params = {
    'year': year,
    'month': month,
    'family': family,
    'store_nbr': store_nbr,
    'onpromotion': onpromotion
}

response = requests.get(url, params=params)
response.json() 

{'sales_baseline_year': {'1': 817.65,
  '2': 1111.44,
  '3': 2399.84,
  '4': 931.13,
  '5': 894.99,
  '6': 1097.47,
  '7': 1277.31,
  '8': 1580.73,
  '9': 1055.24,
  '10': 292.93,
  '11': 920.48,
  '12': 303.68}}

# Train Model (City L1)

In [10]:
X_train, y_train, X_predict, pipe, preprocessor = TrainCity()

In [11]:
X_predict.head(12)

,year,month,city
0,2017,8,Ambato
1,2017,8,Babahoyo
2,2017,8,Cayambe
3,2017,8,Cuenca
4,2017,8,Daule
5,2017,8,El Carmen
6,2017,8,Esmeraldas
7,2017,8,Guaranda
8,2017,8,Guayaquil
9,2017,8,Ibarra


In [12]:
job = joblib.load('model_city.joblib')
job

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['year', 'month', 'city'])])),
                ('linearregression', LinearRegression())])

In [13]:
job.score(X_train,y_train)

0.5692069963894613

In [14]:
p = job.predict(X_predict).tolist()
p[0:12]

[14.3828125,
 13.60546875,
 14.08984375,
 14.5546875,
 13.65625,
 13.125,
 13.5078125,
 13.2890625,
 15.53125,
 13.16796875,
 13.78125,
 12.45703125]

In [15]:
p = [np.around(np.exp(i),decimals=2) for i in p]
p = np.array(p) 
p[0:12]

array([1763502.59,  810550.36, 1315653.09, 2094211.15,  852774.13,
        501320.05,  735137.25,  590699.38, 5560785.87,  523330.64,
        966319.69,  257051.38])

In [16]:
year=['2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017']
month=['8','8','8','8','8','8','8','8','8','8','8','8']
city=['Ambato','Babahoyo', 'Cayambe', 'Cuenca', 'Daule', 'El Carmen', 'Esmeraldas', 'Guaranda', 'Guayaquil', 'Ibarra', 'Latacunga', 'Libertad']

url = 'http://127.0.0.1:8000/predict-city-year'
params = {
    'year': year,
    'month': month,
    'city': city
}

response = requests.get(url, params=params)
response.json() 

{'sales_city_year': {'1': 1763502.59,
  '2': 810550.36,
  '3': 1315653.09,
  '4': 2094211.15,
  '5': 852774.13,
  '6': 501320.05,
  '7': 735137.25,
  '8': 590699.38,
  '9': 5560785.87,
  '10': 523330.64,
  '11': 966319.69,
  '12': 257051.38}}

# Train Model (Store L2)

In [17]:
X_train, y_train, X_predict, pipe, preprocessor = TrainStore()

In [18]:
X_predict.head(12)

,year,month,city,store_nbr
0,2017,8,Ambato,23
1,2017,8,Ambato,50
2,2017,8,Babahoyo,31
3,2017,8,Cayambe,11
4,2017,8,Cuenca,37
5,2017,8,Cuenca,39
6,2017,8,Cuenca,42
7,2017,8,Daule,27
8,2017,8,El Carmen,54
9,2017,8,Esmeraldas,43


In [19]:
job = joblib.load('model_store.joblib')
job

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['year', 'month', 'city',
                                                   'store_nbr'])])),
                ('linearregression', LinearRegression())])

In [20]:
job.score(X_train,y_train)

0.5993945822656733

In [21]:
p = job.predict(X_predict).tolist()
p[0:12]

[13.70703125,
 14.515625,
 14.38671875,
 14.5859375,
 14.23828125,
 14.26953125,
 5.87109375,
 14.30859375,
 13.19921875,
 14.1875,
 14.29296875,
 14.04296875]

In [22]:
p = [np.around(np.exp(i),decimals=2) for i in p]
p = np.array(p) 
p[0:12]

array([8.97197460e+05, 2.01398318e+06, 1.77040475e+06, 2.16068855e+06,
       1.52618429e+06, 1.57463058e+06, 3.54640000e+02, 1.63735673e+06,
       5.39942940e+05, 1.45061767e+06, 1.61197187e+06, 1.25540495e+06])

In [23]:
year=['2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017']
month=['8','8','8','8','8','8','8','8','8','8','8','8']
city=['Ambato','Ambato','Babahoyo','Cayambe','Cuenca','Cuenca','Cuenca','Daule','El Carmen','Esmeraldas','Guaranda','Guayaquil']
store_nbr=['23', '50', '31', '11', '37', '39', '42', '27', '54', '43', '19', '24']

url = 'http://127.0.0.1:8000/predict-store-year'
params = {
    'year': year,
    'month': month,
    'city': city,
    'store_nbr': store_nbr
}

response = requests.get(url, params=params)
response.json()

{'sales_store_year': {'1': 897197.46,
  '2': 2013983.18,
  '3': 1770404.75,
  '4': 2160688.55,
  '5': 1526184.29,
  '6': 1574630.58,
  '7': 354.64,
  '8': 1637356.73,
  '9': 539942.94,
  '10': 1450617.67,
  '11': 1611971.87,
  '12': 1255404.95}}

# Train Model (Family L3)

In [24]:
X_train, y_train, X_predict, pipe, preprocessor = TrainFamily()

In [25]:
X_predict.head(12)

,year,month,city,store_nbr,family
0,2017,8,Ambato,23,AUTOMOTIVE
1,2017,8,Ambato,23,BABY CARE
2,2017,8,Ambato,23,BEAUTY
3,2017,8,Ambato,23,BEVERAGES
4,2017,8,Ambato,23,BOOKS
5,2017,8,Ambato,23,BREAD/BAKERY
6,2017,8,Ambato,23,CELEBRATION
7,2017,8,Ambato,23,CLEANING
8,2017,8,Ambato,23,DAIRY
9,2017,8,Ambato,23,DELI


In [26]:
X_predict.head(12)['family'].tolist()

['AUTOMOTIVE',
 'BABY CARE',
 'BEAUTY',
 'BEVERAGES',
 'BOOKS',
 'BREAD/BAKERY',
 'CELEBRATION',
 'CLEANING',
 'DAIRY',
 'DELI',
 'EGGS',
 'FROZEN FOODS']

In [27]:
job = joblib.load('model_family.joblib')
job

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_tr',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  ['year', 'month', 'city',
                                                   'store_nbr', 'family'])])),
                ('linearregression', LinearRegression())])

In [28]:
job.score(X_train,y_train)

0.6916893767585824

In [29]:
p = job.predict(X_predict).tolist()
p[0:12]

[6.328125,
 0.46484375,
 5.611328125,
 11.791015625,
 -0.12890625,
 10.3271484375,
 4.224609375,
 11.20703125,
 10.646484375,
 9.849609375,
 9.408203125,
 9.044921875]

In [30]:
p = [np.around(np.exp(i),decimals=2) for i in p]
p = np.array(p) 
p[0:12]

array([5.6011000e+02, 1.5900000e+00, 2.7351000e+02, 1.3206053e+05,
       8.8000000e-01, 3.0550870e+04, 6.8350000e+01, 7.3646450e+04,
       4.2044520e+04, 1.8950950e+04, 1.2187950e+04, 8.4753900e+03])

In [31]:
year=['2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017','2017']
month=['8','8','8','8','8','8','8','8','8','8','8','8']
city=['Ambato','Ambato','Ambato','Ambato','Ambato','Ambato','Ambato','Ambato','Ambato','Ambato','Ambato','Ambato']
store_nbr=['23','23','23','23','23','23','23','23','23','23','23','23']
family = ['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS']

url = 'http://127.0.0.1:8000/predict-family-year'
params = {
    'year': year,
    'month': month,
    'city': city,
    'store_nbr': store_nbr,
    'family': family
}

response = requests.get(url, params=params)
response.json()

{'sales_family_year': {'1': 560.11,
  '2': 1.59,
  '3': 273.51,
  '4': 132060.53,
  '5': 0.88,
  '6': 30550.87,
  '7': 68.35,
  '8': 73646.45,
  '9': 42044.52,
  '10': 18950.95,
  '11': 12187.95,
  '12': 8475.39}}